IBES, CRSP, COMPUSTAT data @author Tim Copeland

In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from pandas.tseries.offsets import BDay

Cleaning constituents file

In [6]:
#use compustat to download sp500 constituents from 1970 to 2016.
cons = pd.read_csv('data/cons_70_17.csv', low_memory=False)

In [7]:
#delete duplicate rows
cons = cons[~cons.duplicated()]

#delete rows with empty ticker/company_name
cons = cons.dropna(subset=['co_tic'])

cons = cons.sort_values(['co_tic'])

In [8]:
#fill na's in cons thru with max date (assumed to be nov 1st 2017)
max_date = 20171101.0
cons['thru'] = cons['thru'].fillna(max_date)
cons['thru'] = pd.to_numeric(cons['thru'])
cons['from'] = pd.to_numeric(cons['from'])

In [9]:
cons.head()

,gvkey,gvkeyx,from,thru,conm,tic,co_conm,co_tic,co_cusip
374,10507,3,19640331,20071009.0,S&P 500 Comp-Ltd,I0003,ENERGY FUTURE HOLDINGS CORP,0033A,873168108
1253,12144,3,20020722,20050811.0,S&P 500 Comp-Ltd,I0003,SUNGARD DATA SYSTEMS INC,0139A,867363103
296,2316,3,19640331,19941220.0,S&P 500 Comp-Ltd,I0003,HEXION INC,0141A,428300107
1147,30095,3,19991116,20030925.0,S&P 500 Comp-Ltd,I0003,QUINTILES TRANSNATIONAL CORP,0573B,748767100
484,7148,3,19681231,19750831.0,S&P 500 Comp-Ltd,I0003,MCCRORY CORP,1219A,579865007


In [10]:
cons.describe()

,gvkey,gvkeyx,from,thru
count,1629.000000,1629.0,1.629000e+03,1.629000e+03
mean,25184.907919,3.0,1.985830e+07,2.001934e+07
std,44781.704220,0.0,1.810251e+05,1.518608e+05
min,1010.000000,3.0,1.964033e+07,1.970023e+07
25%,4839.000000,3.0,1.965033e+07,1.988123e+07
50%,8488.000000,3.0,1.985093e+07,2.006040e+07
75%,15084.000000,3.0,2.001080e+07,2.017110e+07
max,316056.000000,3.0,2.017101e+07,2.017110e+07


In [11]:
print(len(cons['co_conm'].unique()))
print(len(cons['co_conm']))

1543
1629


In [12]:
cons.to_csv('data/clean_cons_70_17.csv')

In [13]:
#plaintext of gvkeys
gvkeys = np.sort(cons['gvkey'].unique())
np.savetxt('gvkey.txt', gvkeys[None,:], delimiter="\r\n", fmt="%s")
print(len(gvkeys))

1543


Working with CRSP data

In [14]:
#use gvkeys to download gvkey -> lpermno link table from crsp. Read the resulting table.
Compustat_CRSP_link_table = pd.read_csv('link_tables/Compustat_CRSP_link_table.csv')

In [15]:
print(len(gvkeys) - len(Compustat_CRSP_link_table['gvkey'].unique()))

25


In [16]:
#missing gvkeys
gvkeys = pd.Series(gvkeys)
missing_gvkey = gvkeys[~gvkeys.isin(Compustat_CRSP_link_table['gvkey'].unique())]
missing_gvkey

200       3039
214       3165
319       4199
380       4799
557       6537
752       8430
774       8617
881       9746
1112     13353
1129     14097
1157     15448
1252     27800
1265     28192
1275     28719
1281     29004
1287     29155
1301     29819
1325     31774
1328     32106
1329     32154
1422    122147
1530    186915
1531    186932
1539    265975
1540    266101
dtype: int64

In [17]:
#plaintext of lpermnos
lpermnos = np.sort(Compustat_CRSP_link_table['LPERMNO'].unique())
np.savetxt('lpermno.txt', lpermnos[None,:], delimiter="\r\n", fmt="%s")
print(len(lpermnos))

1727


In [3]:
dtype_dic = {'PERMNO': np.int64,
             'date': np.int64, 
             'NCUSIP': str, 
             'TICKER': object, 
             'COMNAM': str, 
             'CUSIP': object, 
             'DIVAMT': np.float64,
             'BIDLO': np.float64, 
             'ASKHI':np.float64, 
             'PRC': np.float64, #np.int32, 
             'VOL': np.float64, #np.int32, 
             'RET': object, #np.int32,  
             'SHROUT': np.float64, #np.int32, 
             'OPENPRC': np.float64, 
             'NUMTRD': np.float64, #np.int32, 
             'MKTCAP': np.float64
            }

In [23]:
#function that accepts a row of crsp data and checks if this price belongs in the S&P500 at that time or not.
def check_SP500(crsp_entry):
    dt = crsp_entry['date']

    #use lpermno to match with compustat gvkey using Compustat_CRSP_link_table.
    gvkey = Compustat_CRSP_link_table[Compustat_CRSP_link_table['LPERMNO'] == crsp_entry['PERMNO']]['gvkey'].unique()
    if gvkey.size == 0: 
        print(crsp_entry['PERMNO'])
        return False

    #use gvkey to access corresponding date ranges in constituents file
    dt_from = cons[cons['gvkey'] == gvkey[0]]['from']
    dt_thru = cons[cons['gvkey'] == gvkey[0]]['thru']

    #check if actual earnings announcement date is within the (or one of the) date ranges
    for i in range(0,len(dt_from)):
        if dt >= dt_from.values[i] and dt <= dt_thru.values[i]: return True
    
    return False #no match - return false

In [4]:
#save to hdf5 file (Makes it easier to access by slice later on)
hdf = pd.HDFStore('data/crsp.h5', mode = 'a')
for i in pd.read_csv('data/crsp_sp500_1970_2016.csv', chunksize=1000, low_memory = False,dtype=dtype_dic): 
    
    this_chunk = i.copy()
    
    #add a market cap column
    this_chunk['MKTCAP'] = this_chunk['SHROUT'] * 1000 * this_chunk['PRC'] 
    
    #drop rows that are not in the sp500
    #sp500          = this_chunk.apply(check_SP500,axis=1)
    #this_chunk     = this_chunk[sp500 == True]
    hdf.append('df', this_chunk, data_columns=True, format = 'table', min_itemsize={ 'CUSIP': 10, 'NCUSIP': 10, 'TICKER': 10, 'COMNAM' : 35 })

In [19]:
print(sum([len(i['PERMNO'].unique()) for i in pd.read_csv('data/crsp_sp500_1970_2016.csv', chunksize=1000, \
                                                           low_memory = False,dtype=dtype_dic)]))

1243


In [20]:
print(sum([len(i['PERMNO']) for i in pd.read_csv('data/crsp_sp500_1970_2016.csv', chunksize=1000, \
                                                           low_memory = False,dtype=dtype_dic)]))

1048575


In [21]:
print(len(Compustat_CRSP_link_table['LPERMNO'].unique()))

1727


In [24]:
hdf.close()

In [4]:
print(sum([len(i['MKTCAP'].unique()) for i in pd.read_hdf('data/crsp.h5', 'df', mode = 'r',  chunksize=1000, \
                                                           low_memory = False,dtype=dtype_dic)]))

582485


IBES data

In [26]:
#use crsp lpermnos to download crsp lpermno->ibes ticker link table
CRSP_IBES_link_table = pd.read_csv('link_tables/CRSP_IBES_link_table.csv')

In [27]:
print(len(lpermnos) - len(CRSP_IBES_link_table['PERMNO'].unique()))

99


In [28]:
#missing lpermnos (that's a lot!!)
lpermnos = pd.Series(lpermnos)
missing_lpermnos = lpermnos[~lpermnos.isin(CRSP_IBES_link_table['PERMNO'].unique())]
missing_lpermnos

0       10006
17      10276
25      10436
30      10495
32      10524
40      10735
41      10751
52      11041
54      11068
56      11092
70      11543
76      11658
78      11690
80      11746
82      11826
87      11949
104     12124
106     12167
107     12191
110     12343
112     12346
117     12466
128     12669
133     12837
143     13063
154     13311
159     13522
165     13637
179     13964
184     14066
        ...  
893     48119
915     49592
938     51027
971     53357
993     55386
996     56012
1040    59221
1052    59475
1086    61890
1105    63343
1156    67791
1167    69163
1201    75038
1206    75181
1212    75255
1277    77058
1295    77459
1388    80411
1398    80783
1457    83715
1502    85658
1509    85904
1548    87030
1583    88663
1603    89155
1609    89223
1651    90379
1658    90562
1662    90740
1688    91518
Length: 141, dtype: int64

In [29]:
ibtic = np.sort(CRSP_IBES_link_table['TICKER'].unique())
np.savetxt('tic.txt', ibtic[None,:], delimiter="\r\n", fmt="%s")
print(len(ibtic))

1527


In [30]:
estimates = pd.read_csv('ibes/estimates.csv')
estimates.head()

,TICKER,CUSIP,OFTIC,CNAME,ACTDATS,ANALYS,FPI,MEASURE,VALUE,FPEDATS,EXCDATS
0,AA,01381710,AA,ALCOA INC.,20110111,73367,6,EPS,0.57,20110331,20110311
1,AA,01381710,AA,ALCOA INC.,20110606,130416,6,EPS,1.08,20110630,20110708
2,AA,01381710,AA,ALCOA INC.,20111020,146094,6,EPS,0.51,20111231,20120106
3,AA,01381710,AA,ALCOA INC.,20120109,146094,6,EPS,0.21,20111231,20120109
4,AA,01381710,AA,ALCOA INC.,20120111,48907,6,EPS,-0.03,20120331,20120114


In [31]:
estimates.describe()

,ACTDATS,ANALYS,FPI,VALUE,FPEDATS,EXCDATS
count,1.156000e+04,11560.000000,11560.0,11560.000000,1.156000e+04,1.156000e+04
mean,2.013581e+07,88973.570329,6.0,0.758253,2.013564e+07,2.013646e+07
std,1.870992e+04,43842.097733,0.0,1.444451,1.870648e+04,1.870984e+04
min,2.010041e+07,0.000000,6.0,-34.100000,2.010033e+07,2.011010e+07
25%,2.012050e+07,57770.500000,6.0,0.280000,2.012063e+07,2.012061e+07
50%,2.013112e+07,89921.000000,6.0,0.590000,2.013123e+07,2.014011e+07
75%,2.015063e+07,121417.000000,6.0,1.050000,2.015063e+07,2.015072e+07
max,2.017072e+07,187285.000000,6.0,30.710000,2.017093e+07,2.017072e+07


In [32]:
actuals = pd.read_csv('ibes/actuals.csv')
actuals.head()

,TICKER,CUSIP,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,ANNTIMS,ACTDATS,ACTTIMS,VALUE,CURR_ACT
0,A,02742010,A,AMERN MEDIC BLDG,19881231,EPS,QTR,19890401,0:00:00,19890401,0:00:00,0.07,USD
1,A,02742010,A,AMERN MEDIC BLDG,19890331,EPS,QTR,19890517,0:00:00,19890517,0:00:00,0.00,USD
2,A,02742010,A,AMERN MEDIC BLDG,19890630,EPS,QTR,19890816,0:00:00,19890816,0:00:00,-0.09,USD
3,A,02742010,A,AMERN MEDIC BLDG,19890930,EPS,QTR,19891114,0:00:00,19891114,0:00:00,0.00,USD
4,A,02742010,A,AMERN MEDIC BLDG,19891231,EPS,QTR,19900418,0:00:00,19900418,0:00:00,0.00,USD


In [33]:
#drop actuals rows with missing earnings values
actuals = actuals.dropna(axis=0, subset=['VALUE'])

In [34]:
actuals.describe()

,PENDS,ANNDATS,ACTDATS,VALUE
count,5.374100e+04,5.374100e+04,5.374100e+04,53741.000000
mean,2.000218e+07,2.000428e+07,2.000430e+07,0.369432
std,9.398924e+04,9.396942e+04,9.396491e+04,3.315137
min,1.982033e+07,1.982110e+07,1.982110e+07,-386.999800
25%,1.992123e+07,1.993022e+07,1.993022e+07,0.090500
50%,2.000093e+07,2.000102e+07,2.000102e+07,0.280000
75%,2.008073e+07,2.008092e+07,2.008092e+07,0.590000
max,2.017053e+07,2.017063e+07,2.017063e+07,69.320000


In [35]:
#given a date (in YYYYMMDD format), returns quarter number
def quarter_num(date):
    a = datetime.strptime(str(date), '%Y%m%d')
    return (a.month-1)//3

quarter_num('20060801')

2

In [36]:
#add quarternum column to actuals and estimates
estimates['quarternum'] = estimates['FPEDATS'].apply(quarter_num)
actuals['quarternum']   = actuals['ANNDATS'].apply(quarter_num)

In [37]:
#save clean estimates
estimates.to_csv('ibes/clean_estimates.csv')
actuals.to_csv('ibes/clean_actuals.csv')

In [38]:
#analyst forecast and actual earnings dataframes
df_aforecast   = estimates.set_index(['TICKER', 'FPEDATS', 'quarternum', 'ANALYS'])['VALUE']
df_actual_ern = actuals.set_index(['TICKER', 'ANNDATS', 'quarternum'])['VALUE']

In [39]:
df_aforecast.to_csv('ibes/df_aforecast.csv')
df_actual_ern.to_csv('ibes/df_actual_ern.csv')

Generate price distribution dataframe

In [29]:
#clean up missing values in CRSP_IBES_linktable
CRSP_IBES_link_table = CRSP_IBES_link_table.fillna(method='ffill')

In [53]:
class df_handler:
    #actuals = actuals dataframe
    def __init__(self,actuals, X=30): 
        self.actuals = actuals
        self.X = X #denotes range of prices [-X days, +Xdays] from target date
        
    def gen_df(self):
        
        prc_range = [str(i) for i in range(-self.X, self.X+1)]
        col_names = ['TICKER', 'ANNDATS', 'quarternum'] + prc_range
        df = pd.DataFrame(columns = col_names)
        for i in range(0,len(self.actuals['TICKER'])):
            row = self.actuals.iloc[i]
            print(str(row['ANNDATS']) + ' ' + str(row['TICKER']))
            
            #if self.check_SP500(row): 
            prices = self.crsp_prices(row)
            
            if len(prices) !=0:
                data = [row['TICKER'] , row['ANNDATS'] , row['quarternum']] + [i for i in self.crsp_prices(row)]
                tmp = dict(zip(col_names,data))
                df = df.append(tmp, ignore_index = True) 
            
        return df
    #function that accepts a row of actuals data and checks if this earnings belongs in the S&P500 at that time or not.
    def check_SP500(self,actual_entry):
        dt = actual_entry['ANNDATS']

        #match this ibtic to corresponding lpermno via CRSP_IBES_link_table
        ibtic   = actual_entry['TICKER']
        lpermno = CRSP_IBES_link_table[CRSP_IBES_link_table['TICKER'] == ibtic]['PERMNO'].unique()
        if lpermno.size == 0: return False

        #use lpermno to match with compustat gvkey using Compustat_CRSP_link_table. "lpermno[0]" the zero index is because 
        #    .unique( ) returns a numpy array of size 1. We need it as a number instead. Dangerous!!
        gvkey = Compustat_CRSP_link_table[Compustat_CRSP_link_table['LPERMNO'] == lpermno[0]]['gvkey'].unique()
        if gvkey.size == 0: return False

        #use gvkey to access corresponding date ranges in constituents file
        dt_from = cons[cons['gvkey'] == gvkey[0]]['from']
        dt_thru = cons[cons['gvkey'] == gvkey[0]]['thru']

        #check if actual earnings announcement date is within the (or one of the) date ranges
        for i in range(0,len(dt_from)):
            if dt >= dt_from.values[i] and dt <= dt_thru.values[i]: return True

        return False #no match - return false   
    
    #given a row of actuals data, match it to crsp data and generate [-X,X] business days of price
    #data. Prices are normalized such that the earnings date price is 1.
    #     NOTE: there are duplicate dates entries for the same permno. No idea why this is. I just deleted them.
    #This part can definitely be sped up. Some redundant operations..
    #  1. linking ibtics and permnos
    #  2. slicing crsp and manipulating slice.
    def crsp_prices(self,actual_entry):
        dt = actual_entry['ANNDATS']
        X = self.X

        #match this ibtic to corresponding lpermno via CRSP_IBES_link_table
        ibtic   = actual_entry['TICKER']
        lpermno = CRSP_IBES_link_table[CRSP_IBES_link_table['TICKER'] == ibtic]['PERMNO'].unique()
        if lpermno.size == 0: 
            print('MISSING IN LINK TABLE: ' + str(ibtic))
            return []

        #grab crsp prices associated with this ticker

        crsp = pd.concat([pd.read_hdf('data/crsp.h5', 'df', low_memory = False, where='PERMNO in ' + str(i))
                          for i in lpermno])
        if crsp.empty: 
            print('MISSING IN CRSP: ' + str(lpermno[:]))
            return []
                        
        #delete duplicate rows
        crsp = crsp[~crsp.duplicated()]
        
        #delete duplicate dates (Why do these exist in the first place? Is this correct??)
        crsp = crsp[~crsp['date'].duplicated()]
        
        #delete entries with missing dates
        crsp = crsp.dropna(subset=['date'])
        
        #find date of price closest to earnings date
        data     = crsp.sort_values(['date']).reset_index(drop=True)
        prc_date = pd.Index(data['date']).get_loc(dt, method='nearest')
        
        #take slice of dates X business days after dt and X days before dt 
        date_range = [datetime.strptime(str(data['date'][prc_date]), '%Y%m%d') + j * BDay() for j in range(-X, X+1)]
        prices = []
        for i in date_range:
            tar_prc = data[data['date'] == int(i.strftime('%Y%m%d'))]['PRC']

            if tar_prc.empty: prices.append('NaN')
            else:             prices.append(tar_prc.values[0])
                
        #normalize prices such that target date = 1 (ie each price / target date price)
        prices_copy = prices.copy()
        for i in range(0,len(prices)): 
            if type(prices[i]) == str: continue
            else:
                prices[i] = round(prices_copy[i]/prices_copy[X], 4)
        
        return prices

In [56]:
#generate dataframe
a = df_handler(actuals)
df_prc_dist = a.gen_df()

19890401 A
MISSING IN LINK TABLE: A
19890517 A
MISSING IN LINK TABLE: A
19890816 A
MISSING IN LINK TABLE: A
19891114 A
MISSING IN LINK TABLE: A
19900418 A
MISSING IN LINK TABLE: A
19900516 A
MISSING IN LINK TABLE: A
19901115 A
MISSING IN LINK TABLE: A
19901115 A
MISSING IN LINK TABLE: A
19920403 A
MISSING IN LINK TABLE: A
19830421 AA


ValueError: The file 'data/crsp.h5' is already opened, but not in read-only mode (as requested).

In [74]:
df_prc_dist.head()

,TICKER,ANNDATS,quarternum,-30,-29,-28,-27,-26,-25,-24,...,21,22,23,24,25,26,27,28,29,30
0,ABBV,20130426,1,0.8399,0.8325,0.8368,0.8521,0.8397,0.8501,0.8578,...,NaN,0.9902,0.9572,0.9719,0.9313,0.9459,0.9518,0.9389,0.949,0.9599
1,ABBV,20130726,2,0.9647,0.9603,0.9721,0.9609,0.9167,0.9516,0.9353,...,0.9701,0.9518,0.9502,0.9516,0.9513,NaN,0.9509,0.9629,0.9714,0.9799
2,ABBV,20131025,3,0.9105,0.9294,0.9655,0.972,0.9615,0.9704,0.9554,...,0.9819,0.9858,0.9787,NaN,0.9828,0.9815,1.0136,1.0075,1.0093,1.0416
3,ABBV,20140131,0,1.0683,1.068,1.0636,NaN,1.0764,1.0674,1.0768,...,1.0175,1.0479,1.0502,1.0571,1.0453,1.0494,1.0512,1.049,1.0425,1.0402
4,ABBV,20140425,1,1.0421,1.0578,1.0806,1.0808,1.0924,1.0879,1.0452,...,NaN,1.0987,1.1003,1.0995,1.1056,1.102,1.1064,1.1107,1.1254,1.1213


In [75]:
df_prc_dist.describe()

,0
count,5145.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [76]:
df_prc_dist.to_csv('df_prc_dist.csv')

In [19]:
import pandas as pd

ibes_actuals = pd.read_csv('ibes/clean_actuals.csv')
ibes_actuals.ACTDATS = pd.to_datetime(ibes_actuals.ACTDATS,format='%Y%m%d')
ibes_actuals.ANNDATS = pd.to_datetime(ibes_actuals.ANNDATS,format='%Y%m%d')
print(ibes_actuals.columns)
del ibes_actuals['MEASURE']
del ibes_actuals['Unnamed: 0']
del ibes_actuals['PDICITY']
del ibes_actuals['ANNTIMS']
del ibes_actuals['ACTTIMS']
del ibes_actuals['quarternum']
ibes_actuals = ibes_actuals[ibes_actuals.CURR_ACT=='USD']
del ibes_actuals['CURR_ACT']

actual_counts = ibes_actuals.groupby(['CUSIP','ANNDATS']).agg({'VALUE':'count'}).reset_index()

print(actual_counts[actual_counts.VALUE!=1])

#print (ibes_actuals[(ibes_actuals.CUSIP=='92849710') & (ibes_actuals.ANNDATS==pd.to_datetime('2008-09-30'))])
print (ibes_actuals[(ibes_actuals.CUSIP=='92849710') & (ibes_actuals.PENDS=='20080930')])

Index(['Unnamed: 0', 'TICKER', 'CUSIP', 'OFTIC', 'CNAME', 'PENDS', 'MEASURE',
       'PDICITY', 'ANNDATS', 'ANNTIMS', 'ACTDATS', 'ACTTIMS', 'VALUE',
       'CURR_ACT', 'quarternum'],
      dtype='object')
          CUSIP    ANNDATS  VALUE
762    00790310 1987-04-13      2
2798   02742010 1990-11-15      2
3857   03748R10 1997-10-27      2
4560   04040110 1996-05-28      3
4562   04040110 1997-03-27      3
4563   04040110 1997-09-04      2
4825   05361110 1991-02-16      2
5364   05957410 1991-03-20      3
5365   05957410 1991-12-12      2
5372   05957420 1996-10-21      2
5980   07581110 2003-01-27      2
7142   10112110 1998-03-26      2
7620   11132010 2007-01-23      2
8305   12626810 1988-08-30      2
9159   15102010 1991-11-18      6
9162   15102010 1992-11-18      3
9163   15102010 1993-08-19      2
9164   15102010 1994-03-24      2
9166   15102010 1994-11-15      3
9528   15678210 1992-07-20      2
9669   16715510 1992-06-09      4
9670   16715510 1992-10-26      2
9765   170040

c:\users\tcope\appdata\local\continuum\anaconda3\lib\site-packages\pandas\core\ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [30]:
crsp = pd.concat([ i for i in pd.read_hdf('data/crsp.h5', 'df', low_memory = False, chunksize=1000)])

In [31]:
crsp['PERMNO'].unique()

array([10006, 10057, 10078, 10102, 10104, 10107, 10108, 10137, 10138,
       10145, 10147, 10153, 10161, 10225, 10233, 10241, 10276, 10299,
       10321, 10324, 10353, 10364, 10372, 10401, 10436, 10479, 10487,
       10495, 10516, 10524, 10559, 10562, 10604, 10693, 10696, 10725,
       10727, 10751, 10786, 10791, 10823, 10866, 10874, 10890, 10909,
       10942, 10970, 10989, 11041, 11042, 11068, 11081, 11092, 11156,
       11164, 11260, 11295, 11308, 11332, 11340, 11404, 11415, 11447,
       11471, 11543, 11552, 11600, 11607, 11618, 11626, 11658, 11674,
       11690, 11703, 11746, 11754, 11826, 11850, 11896, 11949, 11955,
       11970, 11976, 11981, 11983, 11997, 12044, 12046, 12052, 12060,
       12062, 12067, 12073, 12076, 12079, 12095, 12124, 12140, 12167,
       12308, 12319, 12343, 12345, 12369, 12431, 12456, 12458, 12466,
       12490, 12503, 12511, 12542, 12546, 12558, 12570, 12622, 12626,
       12650, 12669, 12706, 12730, 12749, 12781, 12837, 12872, 12888,
       12933, 12976,